In [1]:
# Exemplo de logs em diferentes níveis
#logging.debug("Esta é uma mensagem de debug")
#logging.info("Ação da aplicação registrada com nível info")
#logging.warning("Este é um aviso")
#logging.error("Este é um erro")
#logging.critical("Este é um erro crítico")

# FIND TRADES

## BIBLIOTECAS

In [2]:
import yfinance as yf
import pandas as pd
import threading
import os
import logging
import shutil

## PARAMETRIZAÇÃO

In [3]:
###PARÂMETROS

#Configurando o log no terminal
logging.basicConfig(
    level=logging.INFO,  # Definir o nível de log
    format='%(asctime)s - %(levelname)s - %(message)s',  # Formato de saída
    force=True
)

#Configurando pastas
path_Data = '../data'
path_Data_STG = path_Data + '/STG'

#Lista de ações para baixar
acoes = ['BBAS3.SA', 'BBSE3.SA', 'TAEE11.SA', 'UNIP6.SA', 'TRPL4.SA']

## AUXILIARES

In [4]:
def limpar_pasta(caminho_pasta):    
    if os.path.exists(caminho_pasta):        
        for arquivo in os.listdir(caminho_pasta):
            caminho_arquivo = os.path.join(caminho_pasta, arquivo)            
            
            if os.path.isfile(caminho_arquivo) or os.path.islink(caminho_arquivo):
                os.unlink(caminho_arquivo)  
            elif os.path.isdir(caminho_arquivo):
                shutil.rmtree(caminho_arquivo)          
    else:
        logging.error(f"A pasta '{caminho_pasta}' não existe.")        

# Função para criar pastas se não existirem
def criar_pasta(caminho):
    if not os.path.exists(caminho):
        os.makedirs(caminho)
        logging.info(f"Pasta '{caminho}' criada.")        
    else:
        limpar_pasta(caminho)
        logging.info(f"Conteúdo da pasta '{caminho}' apagado.")

def baixar_dados_acao(ticker):
    logging.info(f"Baixando dados de {ticker}...")            
    
    path_file = path_Data_STG + '/' + ticker.lower() + '.csv'
    
    acao = yf.Ticker(ticker)
    dados = acao.history(period='2y', interval='60m')  # Você pode alterar o período aqui
    
    dados['CODNEG'] = ticker

    logging.info(f"Salvando dados de {ticker} na stage.")
    dados.to_csv(path_file, sep=';', decimal=',')    
    
    logging.info(f"Dados de {ticker} finalizados.")    

## FLUXO

In [5]:
criar_pasta(path_Data_STG)

threads = []

# Criar e iniciar uma thread por ação
for acao in acoes:
    thread = threading.Thread(target=baixar_dados_acao, args=(acao,))
    threads.append(thread)
    thread.start()

# Esperar todas as threads terminarem
for thread in threads:
    thread.join()

logging.info('Download de todas as ações completo.')    

2024-09-09 14:55:40,093 - INFO - Conteúdo da pasta '../data/STG' apagado.
2024-09-09 14:55:40,094 - INFO - Baixando dados de BBAS3.SA...
2024-09-09 14:55:40,095 - INFO - Baixando dados de BBSE3.SA...
2024-09-09 14:55:40,097 - INFO - Baixando dados de TAEE11.SA...
2024-09-09 14:55:40,098 - INFO - Baixando dados de UNIP6.SA...
2024-09-09 14:55:40,100 - INFO - Baixando dados de TRPL4.SA...
2024-09-09 14:55:41,261 - INFO - Salvando dados de BBSE3.SA na stage.
2024-09-09 14:55:41,345 - INFO - Dados de BBSE3.SA finalizados.
2024-09-09 14:55:41,514 - INFO - Salvando dados de BBAS3.SA na stage.
2024-09-09 14:55:41,584 - INFO - Salvando dados de UNIP6.SA na stage.
2024-09-09 14:55:41,805 - INFO - Dados de BBAS3.SA finalizados.
2024-09-09 14:55:41,875 - INFO - Salvando dados de TAEE11.SA na stage.
2024-09-09 14:55:41,918 - INFO - Salvando dados de TRPL4.SA na stage.
2024-09-09 14:55:42,015 - INFO - Dados de UNIP6.SA finalizados.
2024-09-09 14:55:42,106 - INFO - Dados de TAEE11.SA finalizados.
20